## Variational Inference in Pyro on CIFAR 10

In [27]:
#Paths
path_to_hw2 = 
cifar_path = "cifar-10-batches-py"

#### Google Drive Setup

Ensure that `path_to_hw2` is the correct path to your working directory. It should contain a subdirectory that holds the decompressed files from the CIFAR 10 dataset

In [29]:
#Drive Setup (cuda)

import torch
assert(torch.cuda.is_available())
device = "cuda"

from google.colab import drive
drive.mount('/content/drive')

import os
path_to_batches = path_to_hw2 + "/" + cifar_path
batches = [file for file in sorted(os.listdir(path_to_batches)) if "batch" in file]
batches

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['batches.meta',
 'data_batch_1',
 'data_batch_2',
 'data_batch_3',
 'data_batch_4',
 'data_batch_5',
 'test_batch']

### Data Pre-Processing

#### Unwrapping CIFAR 10 Python Dataset

CIFAR-10 contains 60K 32x32 images divided into 10 clusters. 
- `test_batch` contains 10K samples.
- `data_batch_{1...5}` contains 10K samples each, amounting to total of 50K samples.
- `batches.meta` contains information on the datasets

In [30]:
import pickle
import os
import numpy

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='latin1')
    return dict

data_batches = list()
for i, batch in enumerate(batches):
    batch_path = os.path.join(path_to_batches, batch)
    data_batches.append(unpickle(batch_path))

    print(batch)
    print(data_batches[i].keys())

meta_data = data_batches[0]
training_data = data_batches[1:-1]
test_data = data_batches[-1]

batches.meta
dict_keys(['num_cases_per_batch', 'label_names', 'num_vis'])
data_batch_1
dict_keys(['batch_label', 'labels', 'data', 'filenames'])
data_batch_2
dict_keys(['batch_label', 'labels', 'data', 'filenames'])
data_batch_3
dict_keys(['batch_label', 'labels', 'data', 'filenames'])
data_batch_4
dict_keys(['batch_label', 'labels', 'data', 'filenames'])
data_batch_5
dict_keys(['batch_label', 'labels', 'data', 'filenames'])
test_batch
dict_keys(['batch_label', 'labels', 'data', 'filenames'])


#### CIFAR 10 Databatch Breakdown

The data entry per databatch contains a 10000K samples x 3072 array. 3072 corresponds to each 32x32 color image, 

In [31]:
num_cases_per_batch = meta_data['num_cases_per_batch']
num_vis = meta_data['num_vis']
label_names = meta_data['label_names']
label_names

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

#### Building 10-Cluster Mixed Model for Pyro VI

Installing pyro and building mixed model to perform variational inference

In [35]:
!pip install pyro-ppl

import pyro

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 722 kB 32.6 MB/s 
